In [17]:
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
import random
import pygame
import math
import torch
from torch import nn
from torch import optim
from tensorboardX import SummaryWriter
import time
import load
from models import DynamicTopologyModel
import load_topology_dataset
from load_topology_dataset import MAX_CELL_COUNT, MIN_CELL_COUNT, IMAGE_SIZE
device = "cpu"
EXPERIMENT=4
writer = SummaryWriter('runs/experiment_{}/'.format(EXPERIMENT))

In [18]:
def save_parameters(writer, model, batch_idx):
    for k, v in model.state_dict().items():
        shape = v.shape
        # Don't do this for single weights or biases
        if np.any(np.array(shape) > 1):
            mean = torch.mean(v)
            std_dev = torch.std(v)
            maximum = torch.max(v)
            minimum = torch.min(v)
            writer.add_scalars("{}_{} ".format(k, shape), {"mean": mean,
                                                                    "std_dev": std_dev,
                                                                    "max": maximum,
                                                                    "min": minimum}, batch_idx)
            writer.add_histogram("{}_{}".format(k, shape), v, batch_idx)
            
        else:
            writer.add_scalar("{}_{}".format(k, shape), v.data, batch_idx)

In [19]:
def test_parameters_update(last_parameters_np ,parameters_np):
    for last_param, param in zip(last_parameters_np, parameters_np):
        diff = param - last_param
        #print(last_parameters.max())
        print(diff.max())
        #print(not np.all(diff))


In [20]:
BATCH_SIZE = 1
INPUT_SEQ_LEN = 1
TARGET_SEQ_LEN = 1
LEARNING_RATE = 1e-2
SAVE_VARIABLE_EVERY = 100

# Define Data.
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.33, 0.66
                                                                                                                    ])
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=2)

# Define Model
dynamic_topology_model = DynamicTopologyModel(neighbourhood_hidden_size=32, neighbourhood_cell_count=6,
                     neighbourhood_output_size=16, lstm_hidden_size=32, lstm_layers=2, teacher_forcing_probability=0.5)

parameters = dynamic_topology_model.parameters

# Define optimizer
optimizer = optim.Adam(parameters
                       , lr=LEARNING_RATE)
criterion = nn.MSELoss()
last_parameters_np = list(np.copy(param.detach().numpy()) for param in parameters)

# Train
for batch_idx, data in enumerate(dataloader):
    optimizer.zero_grad()
    reference_cell_input =  data[0]
    reference_cell_present_input = data[1]
    neighbourhood_cell_rel_input = data[2]
    neighbourhood_cell_load_input = data[3]
    reference_cell_target = data[4]
    reference_cell_present_target = data[5]
    neighbourhood_cell_rel_target = data[6]
    decoder_output_seq = dynamic_topology_model.forward(INPUT_SEQ_LEN, TARGET_SEQ_LEN, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
    loss = criterion(decoder_output_seq, reference_cell_target)
    loss.backward()
    optimizer.step()
    if batch_idx == 0:
        parameters = dynamic_topology_model.parameters
        parameters_np = list(param.detach().numpy() for param in parameters)
        test_parameters_update(last_parameters_np, parameters_np)
    if batch_idx % SAVE_VARIABLE_EVERY == 0:
        print(loss)
        inputs = (INPUT_SEQ_LEN, TARGET_SEQ_LEN, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
        #writer.add_graph(dynamic_topology_model, inputs, True)
        writer.add_scalar("loss", loss, batch_idx)
        save_parameters(writer, dynamic_topology_model, batch_idx)

0.009719908
0.009920418
0.009994283
0.009996235
0.009982884
0.009990916
0.009999425
0.0099993795
0.009999931
0.0
0.009999603
0.009999607
0.009999685
0.0
0.00999992
0.00999992
0.0
0.0
0.009999692
0.009999812
0.009999933
0.009999946
0.009999961
0.00999932
0.009999692
0.009999692
0.009999841
0.009999469
0.009999964
0.009999964
0.009999976
0.009999998
tensor(0.8331, grad_fn=<MseLossBackward>)
tensor(0.0024, grad_fn=<MseLossBackward>)
tensor(0.0050, grad_fn=<MseLossBackward>)
tensor(0.0863, grad_fn=<MseLossBackward>)
tensor(0.4361, grad_fn=<MseLossBackward>)
tensor(0.0515, grad_fn=<MseLossBackward>)
tensor(0.3060, grad_fn=<MseLossBackward>)
tensor(0.1600, grad_fn=<MseLossBackward>)
tensor(1.3939, grad_fn=<MseLossBackward>)
tensor(0.2166, grad_fn=<MseLossBackward>)
tensor(0.0629, grad_fn=<MseLossBackward>)
tensor(0.0443, grad_fn=<MseLossBackward>)
tensor(0.0856, grad_fn=<MseLossBackward>)
tensor(0.0181, grad_fn=<MseLossBackward>)
tensor(0.0008, grad_fn=<MseLossBackward>)
tensor(0.0113, grad_

KeyboardInterrupt: 